# Restricted Boltzmann Machines

## Import the appropriate modules

In [1]:
import numpy as np
import theano
import theano.tensor as T
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from theano.tensor.shared_randomstreams import RandomStreams
from util import relu, error_rate, getKaggleMNIST, init_weights
from autoencoder import DNN

## RBM Class

In [4]:
class RBM(object):
    def __init__(self, M, an_id):
        self.M = M
        self.id = an_id
        self.rng = RandomStreams()

    def fit(self, X, learning_rate=0.1, epochs=1, batch_sz=100, show_fig=False):
        N, D = X.shape
        n_batches = N / batch_sz

        W0 = init_weights((D, self.M))
        self.W = theano.shared(W0, 'W_%s' % self.id)
        self.c = theano.shared(np.zeros(self.M), 'c_%s' % self.id)
        self.b = theano.shared(np.zeros(D), 'b_%s' % self.id)
        self.params = [self.W, self.c, self.b]
        self.forward_params = [self.W, self.c]

        # we won't use this to fit the RBM but we will use these for backpropagation later
        # TODO: technically they should be reset before doing backprop
        self.dW = theano.shared(np.zeros(W0.shape), 'dW_%s' % self.id)
        self.dc = theano.shared(np.zeros(self.M), 'dbh_%s' % self.id)
        self.db = theano.shared(np.zeros(D), 'dbo_%s' % self.id)
        self.dparams = [self.dW, self.dc, self.db]
        self.forward_dparams = [self.dW, self.dc]

        X_in = T.matrix('X_%s' % self.id)

        # attach it to the object so it can be used later
        # must be sigmoidal because the output is also a sigmoid
        H = T.nnet.sigmoid(X_in.dot(self.W) + self.c)
        self.hidden_op = theano.function(
            inputs=[X_in],
            outputs=H,
        )

        # we won't use this cost to do any updates
        # but we would like to see how this cost function changes
        # as we do contrastive divergence
        X_hat = self.forward_output(X_in)
        cost = -(X_in * T.log(X_hat) + (1 - X_in) * T.log(1 - X_hat)).sum() / (batch_sz * D)
        cost_op = theano.function(
            inputs=[X_in],
            outputs=cost,
        )

        # do one round of Gibbs sampling to obtain X_sample
        H = self.sample_h_given_v(X_in)
        X_sample = self.sample_v_given_h(H)

        # define the objective, updates, and train function
        objective = T.mean(self.free_energy(X_in)) - T.mean(self.free_energy(X_sample))

        # need to consider X_sample constant because you can't take the gradient of random numbers in Theano
        updates = [(p, p - learning_rate*T.grad(
            objective, p, consider_constant=[X_sample])) for p in self.params]
        train_op = theano.function(
            inputs=[X_in],
            updates=updates,
        )

        costs = []
        print ("training rbm: %s" % self.id)
        for i in range(int(epochs)):
            print ("epoch:", i)
            X = shuffle(X)
            for j in range(int(n_batches)):
                batch = X[j*batch_sz:(j*batch_sz + batch_sz)]
                train_op(batch)
                the_cost = cost_op(X)  # technically we could also get the cost for Xtest here
                print ("j / n_batches:", j, "/", n_batches, "cost:", the_cost)
                costs.append(the_cost)
        if show_fig:
            plt.plot(costs)
            plt.show()

    def free_energy(self, V):
        return  -V.dot(self.b) - T.sum(T.log(1 + T.exp(V.dot(self.W) + self.c)), axis=1)

    def sample_h_given_v(self, V):
        p_h_given_v = T.nnet.sigmoid(V.dot(self.W) + self.c)
        h_sample = self.rng.binomial(size=p_h_given_v.shape, n=1, p=p_h_given_v)
        return h_sample

    def sample_v_given_h(self, H):
        p_v_given_h = T.nnet.sigmoid(H.dot(self.W.T) + self.b)
        v_sample = self.rng.binomial(size=p_v_given_h.shape, n=1, p=p_v_given_h)
        return v_sample

    def forward_hidden(self, X):
        return T.nnet.sigmoid(X.dot(self.W) + self.c)

    def forward_output(self, X):
        Z = self.forward_hidden(X)
        Y = T.nnet.sigmoid(Z.dot(self.W.T) + self.b)
        return Y

    @staticmethod
    def createFromArrays(W, c, b, an_id):
        rbm = AutoEncoder(W.shape[1], an_id)
        rbm.W = theano.shared(W, 'W_%s' % rbm.id)
        rbm.c = theano.shared(c, 'c_%s' % rbm.id)
        rbm.b = theano.shared(b, 'b_%s' % rbm.id)
        rbm.params = [rbm.W, rbm.c, rbm.b]
        rbm.forward_params = [rbm.W, rbm.c]
        return rbm

# Running the model

In [ ]:
Xtrain, Ytrain, Xtest, Ytest = getKaggleMNIST()
dnn = DNN([1000, 750, 500], UnsupervisedModel=RBM)
dnn.fit(Xtrain, Ytrain, Xtest, Ytest, epochs=3)

# we compare with no pretraining in autoencoder.py


training rbm: 0
epoch: 0
j / n_batches: 0 / 410.0 cost: 151.3776596875377
j / n_batches: 1 / 410.0 cost: 146.3203703878674
j / n_batches: 2 / 410.0 cost: 144.0130903707576
j / n_batches: 3 / 410.0 cost: 141.07134167507732
j / n_batches: 4 / 410.0 cost: 131.94823864864358
j / n_batches: 5 / 410.0 cost: 123.84549220651303
j / n_batches: 6 / 410.0 cost: 119.59237113632864
j / n_batches: 7 / 410.0 cost: 123.28984382905686
j / n_batches: 8 / 410.0 cost: 118.12851764677872
j / n_batches: 9 / 410.0 cost: 112.45371540827828
j / n_batches: 10 / 410.0 cost: 110.9067863097084
j / n_batches: 11 / 410.0 cost: 110.19866330603207
j / n_batches: 12 / 410.0 cost: 110.03259367905912
j / n_batches: 13 / 410.0 cost: 112.39211017898299
j / n_batches: 14 / 410.0 cost: 113.12931414052512
j / n_batches: 15 / 410.0 cost: 108.79558595584376
j / n_batches: 16 / 410.0 cost: 107.86973196080658
j / n_batches: 17 / 410.0 cost: 105.73634228737234
j / n_batches: 18 / 410.0 cost: 103.64365574290981
j / n_batches: 19 / 

j / n_batches: 163 / 410.0 cost: 58.432566501640515
j / n_batches: 164 / 410.0 cost: 58.424433173652176
j / n_batches: 165 / 410.0 cost: 58.40426685857158
j / n_batches: 166 / 410.0 cost: 58.471571640986475
j / n_batches: 167 / 410.0 cost: 58.151797982513784
j / n_batches: 168 / 410.0 cost: 58.0907241283887
j / n_batches: 169 / 410.0 cost: 57.93790358601173
j / n_batches: 170 / 410.0 cost: 58.18598168478066
j / n_batches: 171 / 410.0 cost: 57.911122267277584
j / n_batches: 172 / 410.0 cost: 57.60856387974265
j / n_batches: 173 / 410.0 cost: 57.55097580549117
j / n_batches: 174 / 410.0 cost: 57.70576115294002
j / n_batches: 175 / 410.0 cost: 57.55174799777861
j / n_batches: 176 / 410.0 cost: 57.60452997298541
j / n_batches: 177 / 410.0 cost: 57.24867389403472
j / n_batches: 178 / 410.0 cost: 57.28271915531148
j / n_batches: 179 / 410.0 cost: 57.10263897193931
j / n_batches: 180 / 410.0 cost: 57.17591303247873
j / n_batches: 181 / 410.0 cost: 56.704042546667296
j / n_batches: 182 / 410.0

j / n_batches: 324 / 410.0 cost: 49.058878983291045
j / n_batches: 325 / 410.0 cost: 48.95649441385096
j / n_batches: 326 / 410.0 cost: 49.07293571564556
j / n_batches: 327 / 410.0 cost: 48.954846431369475
j / n_batches: 328 / 410.0 cost: 49.094170496547726
j / n_batches: 329 / 410.0 cost: 48.93395157567474
j / n_batches: 330 / 410.0 cost: 48.93236347209904
j / n_batches: 331 / 410.0 cost: 48.96560053306892
j / n_batches: 332 / 410.0 cost: 48.83898159110868
j / n_batches: 333 / 410.0 cost: 48.87194193452536
j / n_batches: 334 / 410.0 cost: 48.71610670115403
j / n_batches: 335 / 410.0 cost: 48.718550777769586
j / n_batches: 336 / 410.0 cost: 48.68512216046242
j / n_batches: 337 / 410.0 cost: 48.612533329026725
j / n_batches: 338 / 410.0 cost: 48.717504906218096
j / n_batches: 339 / 410.0 cost: 48.671644074534036
j / n_batches: 340 / 410.0 cost: 48.64913318784037
j / n_batches: 341 / 410.0 cost: 48.37266792546684
j / n_batches: 342 / 410.0 cost: 48.554328622722714
j / n_batches: 343 / 41

# RBM With TensorFlow

## Import the appropriate modules

In [ ]:
from __future__ import print_function, division
from builtins import range
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from util import getKaggleMNIST
from autoencoder_tf import DNNclass RBM(object):

## RBM Class

In [5]:
class RBM(object):
    def __init__(self, D, M, an_id):
        self.D = D
        self.M = M
        self.id = an_id
        self.build(D, M)

    def set_session(self, session):
        self.session = session

    def build(self, D, M):
        # params
        self.W = tf.Variable(tf.random_normal(shape=(D, M)) * np.sqrt(2.0 / M))
        # note: without limiting variance, you get numerical stability issues
        self.c = tf.Variable(np.zeros(M).astype(np.float32))
        self.b = tf.Variable(np.zeros(D).astype(np.float32))

        # data
        self.X_in = tf.placeholder(tf.float32, shape=(None, D))

        # conditional probabilities
        # NOTE: tf.contrib.distributions.Bernoulli API has changed in Tensorflow v1.2
        V = self.X_in
        p_h_given_v = tf.nn.sigmoid(tf.matmul(V, self.W) + self.c)
        self.p_h_given_v = p_h_given_v # save for later
        # self.rng_h_given_v = tf.contrib.distributions.Bernoulli(
        #     probs=p_h_given_v,
        #     dtype=tf.float32
        # )
        r = tf.random_uniform(shape=tf.shape(p_h_given_v))
        H = tf.to_float(r < p_h_given_v)

        p_v_given_h = tf.nn.sigmoid(
            tf.matmul(H, tf.transpose(self.W)) + self.b)
        # self.rng_v_given_h = tf.contrib.distributions.Bernoulli(
        #     probs=p_v_given_h,
        #     dtype=tf.float32
        # )
        r = tf.random_uniform(shape=tf.shape(p_v_given_h))
        X_sample = tf.to_float(r < p_v_given_h)


        # build the objective
        objective = tf.reduce_mean(
            self.free_energy(self.X_in)) - tf.reduce_mean(self.free_energy(X_sample))
        self.train_op = tf.train.AdamOptimizer(1e-2).minimize(objective)
        # self.train_op = tf.train.GradientDescentOptimizer(1e-3).minimize(objective)

        # build the cost
        # we won't use this to optimize the model parameters
        # just to observe what happens during training
        logits = self.forward_logits(self.X_in)
        self.cost = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(
                labels=self.X_in,
                logits=logits,
            )
        )

    def fit(self, X, epochs=1, batch_sz=100, show_fig=False):
        N, D = X.shape
        n_batches = N // batch_sz

        costs = []
        print("training rbm: %s" % self.id)
        for i in range(epochs):
            print("epoch:", i)
            X = shuffle(X)
            for j in range(n_batches):
                batch = X[j*batch_sz:(j*batch_sz + batch_sz)]
                _, c = self.session.run(
                    (self.train_op, self.cost), feed_dict={self.X_in: batch})
                if j % 10 == 0:
                    print("j / n_batches:", j, "/", n_batches, "cost:", c)
                costs.append(c)
        if show_fig:
            plt.plot(costs)
            plt.show()

    def free_energy(self, V):
        b = tf.reshape(self.b, (self.D, 1))
        first_term = -tf.matmul(V, b)
        first_term = tf.reshape(first_term, (-1,))

        second_term = -tf.reduce_sum(
            # tf.log(1 + tf.exp(tf.matmul(V, self.W) + self.c)),
            tf.nn.softplus(tf.matmul(V, self.W) + self.c),
            axis=1
        )

        return first_term + second_term

    def forward_hidden(self, X):
        return tf.nn.sigmoid(tf.matmul(X, self.W) + self.c)

    def forward_logits(self, X):
        Z = self.forward_hidden(X)
        return tf.matmul(Z, tf.transpose(self.W)) + self.b

    def forward_output(self, X):
        return tf.nn.sigmoid(self.forward_logits(X))

    def transform(self, X):
        # accepts and returns a real numpy array
        # unlike forward_hidden and forward_output
        # which deal with tensorflow variables
        return self.session.run(self.p_h_given_v, feed_dict={self.X_in: X})

## Running the model

In [ ]:
Xtrain, Ytrain, Xtest, Ytest = getKaggleMNIST()

# same as autoencoder_tf.py
Xtrain = Xtrain.astype(np.float32)
Xtest = Xtest.astype(np.float32)
_, D = Xtrain.shape
K = len(set(Ytrain))
dnn = DNN(D, [1000, 750, 500], K, UnsupervisedModel=RBM)
init_op = tf.global_variables_initializer()
with tf.Session() as session:
    session.run(init_op)
    dnn.set_session(session)
    dnn.fit(Xtrain, Ytrain, Xtest, Ytest, pretrain=True, epochs=10)